In [1]:
%matplotlib notebook
%load_ext autoreload
%pwd

'/ocean/projects/asc170022p/mtragoza/lung-project/notebooks'

In [2]:
import sys, os
os.environ['PKG_CONFIG_PATH'] = '/ocean/projects/asc170022p/mtragoza/mambaforge/envs/lung-project/lib/pkgconfig'
import torch
sys.path.append('..')
import project

torch.cuda.is_available()

--------------------------------------------------------------------------
but there are no active ports detected (or Open MPI was unable to use
them).  This is most certainly not what you wanted.  Check your
cables, subnet manager configuration, etc.  The openib BTL will be
ignored for this job.

  Local host: dv002
--------------------------------------------------------------------------


True

In [4]:
%autoreload
emory4dct = project.imaging.Emory4DCT('../data/Emory-4DCT')
emory4dct

In [5]:
import nibabel as nib
from totalsegmentator.python_api import totalsegmentator
from totalsegmentator.libs import combine_masks
help(totalsegmentator)

Help on function totalsegmentator in module totalsegmentator.python_api:

totalsegmentator(input: Union[str, pathlib.Path, nibabel.nifti1.Nifti1Image], output: Union[str, pathlib.Path, NoneType], ml=False, nr_thr_resamp=1, nr_thr_saving=6, fast=False, nora_tag='None', preview=False, task='total', roi_subset=None, statistics=False, radiomics=False, crop_path=None, body_seg=False, force_split=False, output_type='nifti', quiet=False, verbose=False, test=0, skip_saving=False, device='gpu', license_number=None, statistics_exclude_masks_at_border=True, no_derived_masks=False, v1_order=False, fastest=False, roi_subset_robust=None)
    Run TotalSegmentator from within python.
    
    For explanation of the arguments see description of command line
    arguments in bin/TotalSegmentator.
    
    Return: multilabel Nifti1Image



In [6]:
%%time
for case in emory4dct.cases:
    for phase in emory4dct.phases:
        print(case, phase)
        
        image_file = case.nifti_file(phase=phase)
        mask_file = case.mask_file(phase=phase, roi='lung_combined_mask')
        mask_dir = mask_file.parent
        
        totalsegmentator(
            input=image_file,
            output=mask_dir,
            task='total',
            roi_subset=[
                'lung_upper_lobe_right',
                'lung_middle_lobe_right',
                'lung_lower_lobe_right',
                'lung_upper_lobe_left',
                'lung_lower_lobe_left'
            ],
        )
        totalsegmentator(
            input=image_file,
            output=mask_dir,
            task='lung_vessels'
        )
        totalsegmentator(
            input=image_file,
            output=mask_dir,
            task='body'
        )
        mask_nii = combine_masks(
            mask_dir=mask_dir,
            class_type='lung'
        )
        nib.save(mask_nii, mask_file)

Emory4DCTCase('../data/Emory-4DCT', 'Case1Pack', 10 phases) 0

If you use this tool please cite: https://pubs.rsna.org/doi/10.1148/ryai.230024

Generating rough body segmentation...
Resampling...
  Resampled in 0.22s
Predicting...


/ocean/projects/asc170022p/mtragoza/mambaforge/envs/lung-project/lib/python3.10/site-packages/nnunetv2/utilities/plans_handling/plans_handler.py:37: UserWarning: Detected old nnU-Net plans format. Attempting to reconstruct network architecture parameters. If this fails, rerun nnUNetv2_plan_experiment for your dataset. If you use a custom architecture, please downgrade nnU-Net to the version you implemented this or update your implementation + plans.
  warnings.warn("Detected old nnU-Net plans format. Attempting to reconstruct network architecture "
  0%|          | 0/1 [00:00<?, ?it/s]/ocean/projects/asc170022p/mtragoza/mambaforge/envs/lung-project/lib/python3.10/site-packages/torch/nn/modules/conv.py:605: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1712608839953/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv3d(
100%|██████████| 1/1 [00:11<00:00, 11.26s/it]


  Predicted in 19.55s
Resampling...
  cropping from (256, 256, 94) to (256, 193, 93)
Resampling...
  Resampled in 1.13s
Predicting part 1 of 1 ...


/ocean/projects/asc170022p/mtragoza/mambaforge/envs/lung-project/lib/python3.10/site-packages/nnunetv2/utilities/plans_handling/plans_handler.py:37: UserWarning: Detected old nnU-Net plans format. Attempting to reconstruct network architecture parameters. If this fails, rerun nnUNetv2_plan_experiment for your dataset. If you use a custom architecture, please downgrade nnU-Net to the version you implemented this or update your implementation + plans.
  warnings.warn("Detected old nnU-Net plans format. Attempting to reconstruct network architecture "
100%|██████████| 4/4 [00:01<00:00,  3.55it/s]


  Predicted in 8.52s
Resampling...
Saving segmentations...


...

  Predicted in 13.39s


100%|██████████| 1/1 [00:00<00:00,  2.49it/s]


Resampling...
Saving segmentations...
Creating body_trunc.nii.gz
Creating body_extremities.nii.gz


  0%|          | 0/2 [00:00<?, ?it/s]

  Saved in 1.04s
Creating body.nii.gz
Creating skin.nii.gz
CPU times: user 1h 13min 27s, sys: 10min 31s, total: 1h 23min 59s
Wall time: 3h 52min 51s


In [7]:
case = emory4dct[0]
case.load_niftis()
case.load_masks()

Loading ../data/Emory-4DCT/Case1Pack/NIFTI/case1_T00.nii.gz
Loading ../data/Emory-4DCT/Case1Pack/NIFTI/case1_T10.nii.gz
Loading ../data/Emory-4DCT/Case1Pack/NIFTI/case1_T20.nii.gz
Loading ../data/Emory-4DCT/Case1Pack/NIFTI/case1_T30.nii.gz
Loading ../data/Emory-4DCT/Case1Pack/NIFTI/case1_T40.nii.gz
Loading ../data/Emory-4DCT/Case1Pack/NIFTI/case1_T50.nii.gz
Loading ../data/Emory-4DCT/Case1Pack/NIFTI/case1_T60.nii.gz
Loading ../data/Emory-4DCT/Case1Pack/NIFTI/case1_T70.nii.gz
Loading ../data/Emory-4DCT/Case1Pack/NIFTI/case1_T80.nii.gz
Loading ../data/Emory-4DCT/Case1Pack/NIFTI/case1_T90.nii.gz
Loading ../data/Emory-4DCT/Case1Pack/TotalSegment/case1_T00/lung_combined_mask.nii.gz
Loading ../data/Emory-4DCT/Case1Pack/TotalSegment/case1_T10/lung_combined_mask.nii.gz
Loading ../data/Emory-4DCT/Case1Pack/TotalSegment/case1_T20/lung_combined_mask.nii.gz
Loading ../data/Emory-4DCT/Case1Pack/TotalSegment/case1_T30/lung_combined_mask.nii.gz
Loading ../data/Emory-4DCT/Case1Pack/TotalSegment/case1_

In [9]:
project.visual.view(case.mask)

interactive(children=(SelectionSlider(description='phase', options=((0, 0), (10, 1), (20, 2), (30, 3), (40, 4)…